# 2. API の保護

API Gateway の接続に API キーを要求するようセットアップしてみましょう。

## 2.1. 変数・初期値の設定

これまで利用してきた環境変数を再設定します。

In [ ]:
source /home/jupyter/config/.env
echo "Gateway name: ${gateway_name}"

応答例）
```text
Gateway name: prod-gateway
```

## 2.2. API キーを要求するゲートウェイへの更新

[API キーを securityDefinitions に指定した API 定義](resources/v3.yaml) を作成し、それを構成 v3 として登録します。

<img src="notebook-assets/2-2-config.png" width=70%>

In [ ]:
cat resources/v3.yaml | grep -A 4 securityDefinitions
echo "---"
cat resources/v3.yaml | grep -A 1 security:

API キーを必要とする API に `security` を指定し、この設定で構成 v3 を作成します。

In [ ]:
gcloud beta api-gateway api-configs create v3 \
    --api="${api_name}" --openapi-spec=resources/v3.yaml \
    --backend-auth-service-account="${service_account}"

応答例）
```text
Waiting for API Config [v3] to be created for API [my-apis]...done. 
```

新しく作成した v3 の構成をゲートウェイに適用し、その挙動を変えてみます。

<img src="notebook-assets/2-2-gateway.png" width=70%>

In [ ]:
gcloud beta api-gateway gateways update "${gateway_name}" --location=asia-east1 \
    --api="${api_name}" --api-config=v3

応答例）
```text
Waiting for API Gateway [prod-gateway] to be updated...done.
```

認証が必要だとエラー応答になることを確認してみましょう。

In [ ]:
curl -i -XGET "https://$( gcloud beta api-gateway gateways describe ${gateway_name} \
    --location=asia-east1 --format 'value(defaultHostname)' )"

応答例）
```text
HTTP/2 401
content-type: application/json
..

{"message":"UNAUTHENTICATED:Method doesn't allow unregistered callers (callers without established identity). Please use API Key or other form of API consumer identity to call this API.","code":401}
```

## 2.3. API の有効化

以下のように API キーを利用することで、正常にアクセスできることを見ていきます。

<img src="notebook-assets/2-3-gateway.png" width=70%>

まず、Google Cloud のその他の API 同様に、設定やキー生成を利用できるよう本 API を有効化します。

In [ ]:
gcloud services enable "$( gcloud beta api-gateway apis describe "${api_name}" --format 'value(managedService)' )"

以下の結果で表示されるコンソールから API キーを作成します。

In [ ]:
echo "https://console.cloud.google.com/apis/api/$( gcloud beta api-gateway apis describe "${api_name}" \
    --format 'value(managedService)' )/credentials?project=${PROJECT_ID}"

　

<img src="notebook-assets/2-3-key.png" width=50%>

作成した API キーを環境変数 `api_key` とすれば、以下のような要求が正常に応答することが確認できます。

In [ ]:
api_key=<your-api-key>
curl -s "https://$( gcloud beta api-gateway gateways describe ${gateway_name} \
    --location=asia-east1 --format 'value(defaultHostname)' )?key=${api_key}" \
    | jq -r '.req.remote_addr'

応答例）
```text
10.123.12.123:21350
```

[次へ: 5-teardown-resources.ipynb](5-teardown-resources.ipynb)